In [1]:
import gc

# Function to clear memory
def clear_memory():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

In [2]:
import pandas as pd 
import numpy as np

In [3]:
!pip install opencv-python

In [4]:
!pip install einops

In [5]:
!pip install remotezip

In [6]:
!pip install seaborn

In [7]:
import tqdm
import random
import pathlib
import itertools
import collections

#import cv2
import einops
import numpy as np
import remotezip as rz
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras import layers

2024-08-04 22:49:35.770001: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Load in DataFrame

In [8]:
df = pd.read_csv('df_final.csv')
df.head(3)

,Index,Link,Label,Text,FileName,Transcription,sort_key
0,1.0,https://www.tiktok.com/@1tashyat/video/7359361...,1,"""ST Anselm College. She's a Republican. This w...",1_mp4_trial_2.json,ST Anselm College. She's a Republican. This wi...,1
1,2.0,https://www.tiktok.com/@monkeman317/video/7357...,1,"""Said though, that if you did run for presiden...",2_mp4_trial_2.json,"Said though, that if you did run for president...",2
2,3.0,https://www.tiktok.com/@bwtgrils_/video/736257...,1,Yeah.',3_mp4_trial_2.json,Yeah.,3


In [9]:
replacement_mapping = {'Anti - Biden': 1, 'Pro - Biden': 2, 'Neutral':0, 'Anti-Biden': 1, 'Pro-Biden': 2, 'Anti-biden': 1, 'anti-Biden': 1, 'Anti-Biden ': 1, 'Neutral ':0, 'Po-Biden': 2, 'Pro-biden': 2, 'Pro-Biden ': 2}

df = df.replace(replacement_mapping)
df.head(3)

,Index,Link,Label,Text,FileName,Transcription,sort_key
0,1.0,https://www.tiktok.com/@1tashyat/video/7359361...,1,"""ST Anselm College. She's a Republican. This w...",1_mp4_trial_2.json,ST Anselm College. She's a Republican. This wi...,1
1,2.0,https://www.tiktok.com/@monkeman317/video/7357...,1,"""Said though, that if you did run for presiden...",2_mp4_trial_2.json,"Said though, that if you did run for president...",2
2,3.0,https://www.tiktok.com/@bwtgrils_/video/736257...,1,Yeah.',3_mp4_trial_2.json,Yeah.,3


In [29]:
#df['Index'].fillna(0, inplace=True)
#df = df[df['Index'] != 0.0]

In [10]:
df['Video Name'] = df['Index'].apply(lambda x: f'video{int(x)}.mp4')
df.head(3)

,Index,Link,Label,Text,FileName,Transcription,sort_key,Video Name
0,1.0,https://www.tiktok.com/@1tashyat/video/7359361...,1,"""ST Anselm College. She's a Republican. This w...",1_mp4_trial_2.json,ST Anselm College. She's a Republican. This wi...,1,video1.mp4
1,2.0,https://www.tiktok.com/@monkeman317/video/7357...,1,"""Said though, that if you did run for presiden...",2_mp4_trial_2.json,"Said though, that if you did run for president...",2,video2.mp4
2,3.0,https://www.tiktok.com/@bwtgrils_/video/736257...,1,Yeah.',3_mp4_trial_2.json,Yeah.,3,video3.mp4


In [11]:
base_dir = 'path/to/videos/'
video_paths = df['Video Name'].apply(lambda x: base_dir + x).tolist()
labels = df['Label'].tolist()

# Download Train/Test Videos

In [15]:
import boto3
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Define the bucket name and a placeholder function for downloading
bucket_name = 'sagemaker-us-east-2-058264083825'

def download_video_from_s3(bucket_name, object_key, download_path):
    # Placeholder function: Replace with your actual download logic
    print(f"Downloading {object_key} to {download_path}")
    # Your download code here

# Download training videos and match with labels
train_download_paths = []
train_labels = []
for idx, row in train_df.iterrows():
    video_name = row['Video Name']
    label = row['Label']
    download_path = f'train_video_{idx}.mp4'
    download_video_from_s3(bucket_name, video_name, download_path)
    train_download_paths.append(download_path)
    train_labels.append(label)

# Download test videos and match with labels
test_download_paths = []
test_labels = []
for idx, row in test_df.iterrows():
    video_name = row['Video Name']
    label = row['Label']
    download_path = f'test_video_{idx}.mp4'
    download_video_from_s3(bucket_name, video_name, download_path)
    test_download_paths.append(download_path)
    test_labels.append(label)

# Create new DataFrames for the downloaded paths and labels
train_df_downloaded = pd.DataFrame({
    'video_path': train_download_paths,
    'label': train_labels
})

test_df_downloaded = pd.DataFrame({
    'video_path': test_download_paths,
    'label': test_labels
})

# Output the DataFrames
print("Training DataFrame:")
print(train_df_downloaded)
print("\nTest DataFrame:")
print(test_df_downloaded)

Training DataFrame:
              video_path  label
0    train_video_796.mp4      2
1    train_video_101.mp4      1
2    train_video_798.mp4      2
3    train_video_530.mp4      2
4    train_video_430.mp4      0
..                   ...    ...
679   train_video_71.mp4      1
680  train_video_106.mp4      1
681  train_video_270.mp4      1
682  train_video_435.mp4      0
683  train_video_102.mp4      1

[684 rows x 2 columns]

Test DataFrame:
             video_path  label
0     test_video_66.mp4      1
1    test_video_434.mp4      0
2    test_video_198.mp4      1
3    test_video_212.mp4      1
4    test_video_801.mp4      2
..                  ...    ...
166  test_video_211.mp4      1
167  test_video_259.mp4      1
168    test_video_7.mp4      1
169  test_video_281.mp4      1
170  test_video_155.mp4      1

[171 rows x 2 columns]


# Extract Features from BERT Model

In [12]:
# This command installs the necessary OpenGL library for OpenCV
!sudo apt-get update
!sudo apt-get install -y libgl1-mesa-glx

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease 
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease    
Fetched 257 kB in 1s (324 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libgl1-mesa-glx is already the newest version (23.0.4-0ubuntu1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.


In [13]:
import cv2

In [14]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from tqdm import tqdm

# Load model and tokenizer
model_name = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define Dataset class
class SentimentDataset(Dataset):
    def __init__(self, texts, tokenizer):
        self.texts = texts
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        encoded_text = self.tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors='pt')
        encoded_text = {key: value.squeeze(0) for key, value in encoded_text.items()}
        return encoded_text

# Function to extract features in batches
def extract_features(texts, batch_size=16):
    dataset = SentimentDataset(texts, tokenizer)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

    model.eval()
    features = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Extracting Features"):
            inputs = {key: value.to(device) for key, value in batch.items()}
            outputs = model(**inputs)
            features.append(outputs.logits.detach().cpu().numpy())

            # Clear unnecessary data
            del inputs, outputs
            torch.cuda.empty_cache()

    features = np.concatenate(features, axis=0)
    return features

# Assuming df is your DataFrame with 'Text' column
texts = df['Text'].astype(str).tolist()  # Ensure texts are strings
text_features = extract_features(texts)

# Add extracted features to the DataFrame and ensure correct shape
df['Text_Features'] = list(text_features)

# Ensure all text features have the correct shape
df['Text_Features'] = df['Text_Features'].apply(lambda x: x.reshape(-1))

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Extracting Features: 100%|██████████| 54/54 [01:58<00:00,  2.19s/it]


# Hybrid Model: Run #1

In [20]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential
import einops
import cv2
import numpy as np

# Define dimensions
HEIGHT = 112
WIDTH = 112
N_FRAMES = 3
BATCH_SIZE = 1
MAX_TEXT_FEATURES = 3  # This should match the number of classes from the LLM

# Define Conv2Plus1D layer
class Conv2Plus1D(layers.Layer):
    def __init__(self, filters, kernel_size, padding):
        super().__init__()
        self.seq = Sequential([
            layers.Conv3D(filters=filters, kernel_size=(1, kernel_size[1], kernel_size[2]), padding=padding),
            layers.Conv3D(filters=filters, kernel_size=(kernel_size[0], 1, 1), padding=padding)
        ])

    def call(self, x):
        return self.seq(x)

# Define Project layer
class Project(layers.Layer):
    def __init__(self, filters):
        super().__init__()
        self.conv = layers.Conv3D(filters, kernel_size=1)

    def call(self, x):
        return self.conv(x)

# Define ResidualMain layer
class ResidualMain(layers.Layer):
    def __init__(self, filters, kernel_size):
        super().__init__()
        self.seq = Sequential([
            Conv2Plus1D(filters=filters, kernel_size=kernel_size, padding='same'),
            layers.LayerNormalization(),
            layers.ReLU(),
            Conv2Plus1D(filters=filters, kernel_size=kernel_size, padding='same'),
            layers.LayerNormalization()
        ])

    def call(self, x):
        return self.seq(x)

# Add residual block
def add_residual_block(input, filters, kernel_size):
    out = ResidualMain(filters, kernel_size)(input)
    res = input
    if out.shape[-1] != input.shape[-1]:
        res = Project(out.shape[-1])(res)
    return layers.add([res, out])

# Resize video frames
class ResizeVideo(layers.Layer):
    def __init__(self, height, width):
        super().__init__()
        self.height = height
        self.width = width
        self.resizing_layer = layers.Resizing(self.height, self.width)

    def call(self, video):
        old_shape = einops.parse_shape(video, 'b t h w c')
        images = einops.rearrange(video, 'b t h w c -> (b t) h w c')
        images = self.resizing_layer(images)
        videos = einops.rearrange(images, '(b t) h w c -> b t h w c', t=old_shape['t'])
        return videos

# Video input model
def build_video_model(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = ResizeVideo(HEIGHT, WIDTH)(inputs)
    x = add_residual_block(x, filters=32, kernel_size=(3, 3, 3))
    x = layers.MaxPooling3D(pool_size=(1, 2, 2))(x)
    x = add_residual_block(x, filters=64, kernel_size=(3, 3, 3))
    x = layers.MaxPooling3D(pool_size=(1, 2, 2))(x)
    x = layers.GlobalAveragePooling3D()(x)
    return inputs, x

# Combined model
def build_combined_model(video_input_shape, text_feature_shape):
    video_inputs, video_features = build_video_model(video_input_shape)
    text_inputs = tf.keras.Input(shape=text_feature_shape)

    combined_features = tf.keras.layers.concatenate([video_features, text_inputs])
    outputs = tf.keras.layers.Dense(3, activation='softmax')(combined_features)  # Assuming 3 classes for classification
    model = tf.keras.Model(inputs=[video_inputs, text_inputs], outputs=outputs)
    return model

# Example usage
video_input_shape = (N_FRAMES, HEIGHT, WIDTH, 3)
text_feature_shape = (MAX_TEXT_FEATURES,)

model = build_combined_model(video_input_shape, text_feature_shape)
model.summary()

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# FrameGenerator class to preprocess video frames
class FrameGenerator:
    def __init__(self, paths, labels, features, n_frames, frame_size=(HEIGHT, WIDTH), training=False):
        self.paths = paths
        self.labels = labels
        self.features = features
        self.n_frames = n_frames
        self.frame_size = frame_size
        self.training = training

    def __call__(self):
        for video_path, label, feature in zip(self.paths, self.labels, self.features):
            cap = cv2.VideoCapture(video_path)
            frames = []
            while len(frames) < self.n_frames:
                ret, frame = cap.read()
                if not ret:
                    break
                frame = cv2.resize(frame, self.frame_size)  # Resize frames to smaller size
                frame = frame / 255.0  # Normalize
                frames.append(frame)
            cap.release()
            if len(frames) < self.n_frames:
                frames.extend([np.zeros(self.frame_size + (3,))] * (self.n_frames - len(frames)))  # Pad with zeros
            frames = np.array(frames)

            yield (frames, feature), label

output_signature = (
    (tf.TensorSpec(shape=(N_FRAMES, HEIGHT, WIDTH, 3), dtype=tf.float32), tf.TensorSpec(shape=(MAX_TEXT_FEATURES,), dtype=tf.float32)),
    tf.TensorSpec(shape=(), dtype=tf.int16)
)

def create_dataset(paths, labels, features, n_frames, batch_size, frame_size, training=False):
    dataset = tf.data.Dataset.from_generator(
        FrameGenerator(paths, labels, features, n_frames, frame_size, training=training),
        output_signature=output_signature
    )
    dataset = dataset.batch(batch_size)
    return dataset

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_paths = train_df['Video Name'].tolist()
train_labels = train_df['Label'].tolist()
train_features = np.array(train_df['Text_Features'].tolist())

val_paths = test_df['Video Name'].tolist()
val_labels = test_df['Label'].tolist()
val_features = np.array(test_df['Text_Features'].tolist())

train_ds = create_dataset(train_paths, train_labels, train_features, N_FRAMES, BATCH_SIZE, frame_size=(HEIGHT, WIDTH), training=True)
val_ds = create_dataset(val_paths, val_labels, val_features, N_FRAMES, BATCH_SIZE, frame_size=(HEIGHT, WIDTH))

import gc
gc.collect()
if tf.config.list_physical_devices('GPU'):
    tf.keras.backend.clear_session()
    tf.compat.v1.reset_default_graph()

# Fit the model
history = model.fit(train_ds, epochs=10, validation_data=val_ds)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 3, 112, 112, 3)]     0         []                            
                                                                                                  
 resize_video_2 (ResizeVide  (None, 3, 112, 112, 3)       0         ['input_5[0][0]']             
 o)                                                                                               
                                                                                                  
 project_4 (Project)         (None, 3, 112, 112, 32)      128       ['resize_video_2[0][0]']      
                                                                                                  
 residual_main_4 (ResidualM  (None, 3, 112, 112, 32)      16480     ['resize_video_2[0][0]']

In [15]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential
import einops
import cv2
import numpy as np

# Define dimensions
HEIGHT = 112
WIDTH = 112
N_FRAMES = 3
BATCH_SIZE = 1
MAX_TEXT_FEATURES = 3  # This should match the number of classes from the LLM

# Define Conv2Plus1D layer
class Conv2Plus1D(layers.Layer):
    def __init__(self, filters, kernel_size, padding):
        super().__init__()
        self.seq = Sequential([
            layers.Conv3D(filters=filters, kernel_size=(1, kernel_size[1], kernel_size[2]), padding=padding),
            layers.Conv3D(filters=filters, kernel_size=(kernel_size[0], 1, 1), padding=padding)
        ])

    def call(self, x):
        return self.seq(x)

# Define Project layer
class Project(layers.Layer):
    def __init__(self, filters):
        super().__init__()
        self.conv = layers.Conv3D(filters, kernel_size=1)

    def call(self, x):
        return self.conv(x)

# Define ResidualMain layer
class ResidualMain(layers.Layer):
    def __init__(self, filters, kernel_size):
        super().__init__()
        self.seq = Sequential([
            Conv2Plus1D(filters=filters, kernel_size=kernel_size, padding='same'),
            layers.LayerNormalization(),
            layers.ReLU(),
            Conv2Plus1D(filters=filters, kernel_size=kernel_size, padding='same'),
            layers.LayerNormalization()
        ])

    def call(self, x):
        return self.seq(x)

# Add residual block
def add_residual_block(input, filters, kernel_size):
    out = ResidualMain(filters, kernel_size)(input)
    res = input
    if out.shape[-1] != input.shape[-1]:
        res = Project(out.shape[-1])(res)
    return layers.add([res, out])

# Resize video frames
class ResizeVideo(layers.Layer):
    def __init__(self, height, width):
        super().__init__()
        self.height = height
        self.width = width
        self.resizing_layer = layers.Resizing(self.height, self.width)

    def call(self, video):
        old_shape = einops.parse_shape(video, 'b t h w c')
        images = einops.rearrange(video, 'b t h w c -> (b t) h w c')
        images = self.resizing_layer(images)
        videos = einops.rearrange(images, '(b t) h w c -> b t h w c', t=old_shape['t'])
        return videos

# Video input model
def build_video_model(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = ResizeVideo(HEIGHT, WIDTH)(inputs)
    x = add_residual_block(x, filters=32, kernel_size=(3, 3, 3))
    x = layers.MaxPooling3D(pool_size=(1, 2, 2))(x)
    x = add_residual_block(x, filters=64, kernel_size=(3, 3, 3))
    x = layers.MaxPooling3D(pool_size=(1, 2, 2))(x)
    x = layers.GlobalAveragePooling3D()(x)
    return inputs, x

# Combined model
def build_combined_model(video_input_shape, text_feature_shape):
    video_inputs, video_features = build_video_model(video_input_shape)
    text_inputs = tf.keras.Input(shape=text_feature_shape)

    combined_features = tf.keras.layers.concatenate([video_features, text_inputs])
    outputs = tf.keras.layers.Dense(3, activation='softmax')(combined_features)  # Assuming 3 classes for classification
    model = tf.keras.Model(inputs=[video_inputs, text_inputs], outputs=outputs)
    return model

# Example usage
video_input_shape = (N_FRAMES, HEIGHT, WIDTH, 3)
text_feature_shape = (MAX_TEXT_FEATURES,)

model = build_combined_model(video_input_shape, text_feature_shape)
model.summary()

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# FrameGenerator class to preprocess video frames
class FrameGenerator:
    def __init__(self, paths, labels, features, n_frames, frame_size=(HEIGHT, WIDTH), training=False):
        self.paths = paths
        self.labels = labels
        self.features = features
        self.n_frames = n_frames
        self.frame_size = frame_size
        self.training = training

    def __call__(self):
        for video_path, label, feature in zip(self.paths, self.labels, self.features):
            cap = cv2.VideoCapture(video_path)
            frames = []
            while len(frames) < self.n_frames:
                ret, frame = cap.read()
                if not ret:
                    break
                frame = cv2.resize(frame, self.frame_size)  # Resize frames to smaller size
                frame = frame / 255.0  # Normalize
                frames.append(frame)
            cap.release()
            if len(frames) < self.n_frames:
                frames.extend([np.zeros(self.frame_size + (3,))] * (self.n_frames - len(frames)))  # Pad with zeros
            frames = np.array(frames)

            yield (frames, feature), label

output_signature = (
    (tf.TensorSpec(shape=(N_FRAMES, HEIGHT, WIDTH, 3), dtype=tf.float32), tf.TensorSpec(shape=(MAX_TEXT_FEATURES,), dtype=tf.float32)),
    tf.TensorSpec(shape=(), dtype=tf.int16)
)

def create_dataset(paths, labels, features, n_frames, batch_size, frame_size, training=False):
    dataset = tf.data.Dataset.from_generator(
        FrameGenerator(paths, labels, features, n_frames, frame_size, training=training),
        output_signature=output_signature
    )
    dataset = dataset.batch(batch_size)
    return dataset

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_paths = train_df['Video Name'].tolist()
train_labels = train_df['Label'].tolist()
train_features = np.array(train_df['Text_Features'].tolist())

val_paths = test_df['Video Name'].tolist()
val_labels = test_df['Label'].tolist()
val_features = np.array(test_df['Text_Features'].tolist())

train_ds = create_dataset(train_paths, train_labels, train_features, N_FRAMES, BATCH_SIZE, frame_size=(HEIGHT, WIDTH), training=True)
val_ds = create_dataset(val_paths, val_labels, val_features, N_FRAMES, BATCH_SIZE, frame_size=(HEIGHT, WIDTH))

import gc
gc.collect()
if tf.config.list_physical_devices('GPU'):
    tf.keras.backend.clear_session()
    tf.compat.v1.reset_default_graph()

# Fit the model
history = model.fit(train_ds, epochs=10, validation_data=val_ds)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 3, 112, 112, 3)]     0         []                            
                                                                                                  
 resize_video (ResizeVideo)  (None, 3, 112, 112, 3)       0         ['input_1[0][0]']             
                                                                                                  
 project (Project)           (None, 3, 112, 112, 32)      128       ['resize_video[0][0]']        
                                                                                                  
 residual_main (ResidualMai  (None, 3, 112, 112, 32)      16480     ['resize_video[0][0]']        
 n)                                                                                           

# Hybrid Model: Run 2

In [19]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential
import einops
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Define dimensions
HEIGHT = 112
WIDTH = 112
N_FRAMES = 3
BATCH_SIZE = 1
MAX_TEXT_FEATURES = 3  # This should match the number of classes from the LLM

# Define Conv2Plus1D layer
class Conv2Plus1D(layers.Layer):
    def __init__(self, filters, kernel_size, padding):
        super().__init__()
        self.seq = Sequential([
            layers.Conv3D(filters=filters, kernel_size=(1, kernel_size[1], kernel_size[2]), padding=padding),
            layers.Conv3D(filters=filters, kernel_size=(kernel_size[0], 1, 1), padding=padding)
        ])

    def call(self, x):
        return self.seq(x)

# Define Project layer
class Project(layers.Layer):
    def __init__(self, filters):
        super().__init__()
        self.conv = layers.Conv3D(filters, kernel_size=1)

    def call(self, x):
        return self.conv(x)

# Define ResidualMain layer
class ResidualMain(layers.Layer):
    def __init__(self, filters, kernel_size, dropout_rate=0.3, l2_strength=0.01):
        super().__init__()
        self.seq = Sequential([
            Conv2Plus1D(filters=filters, kernel_size=kernel_size, padding='same'),
            layers.LayerNormalization(),
            layers.ReLU(),
            layers.Dropout(dropout_rate),
            Conv2Plus1D(filters=filters, kernel_size=kernel_size, padding='same'),
            layers.LayerNormalization(),
            layers.Dropout(dropout_rate)
        ])

    def call(self, x):
        return self.seq(x)

# Add residual block
def add_residual_block(input, filters, kernel_size):
    out = ResidualMain(filters, kernel_size)(input)x
    res = input
    if out.shape[-1] != input.shape[-1]:
        res = Project(out.shape[-1])(res)
    return layers.add([res, out])

# Resize video frames
class ResizeVideo(layers.Layer):
    def __init__(self, height, width):
        super().__init__()
        self.height = height
        self.width = width
        self.resizing_layer = layers.Resizing(self.height, self.width)

    def call(self, video):
        old_shape = einops.parse_shape(video, 'b t h w c')
        images = einops.rearrange(video, 'b t h w c -> (b t) h w c')
        images = self.resizing_layer(images)
        videos = einops.rearrange(images, '(b t) h w c -> b t h w c', t=old_shape['t'])
        return videos

# Data augmentation for video frames
class VideoDataAugmentation(layers.Layer):
    def __init__(self, frame_size):
        super().__init__()
        self.frame_size = frame_size
        self.augmentation_layer = Sequential([
            layers.RandomFlip("horizontal"),
            layers.RandomRotation(0.1),
            layers.RandomZoom(0.1),
            layers.RandomCrop(self.frame_size[0], self.frame_size[1])
        ])

    def call(self, video):
        old_shape = einops.parse_shape(video, 'b t h w c')
        images = einops.rearrange(video, 'b t h w c -> (b t) h w c')
        images = self.augmentation_layer(images)
        videos = einops.rearrange(images, '(b t) h w c -> b t h w c', t=old_shape['t'])
        return videos

# Video input model
def build_video_model(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = ResizeVideo(HEIGHT, WIDTH)(inputs)
    x = VideoDataAugmentation((HEIGHT, WIDTH))(x)  # Apply augmentation
    x = add_residual_block(x, filters=32, kernel_size=(3, 3, 3))
    x = layers.MaxPooling3D(pool_size=(1, 2, 2))(x)
    x = add_residual_block(x, filters=64, kernel_size=(3, 3, 3))
    x = layers.MaxPooling3D(pool_size=(1, 2, 2))(x)
    x = add_residual_block(x, filters=128, kernel_size=(3, 3, 3))  # New block
    x = layers.GlobalAveragePooling3D()(x)
    return inputs, x

# Combined model
def build_combined_model(video_input_shape, text_feature_shape):
    video_inputs, video_features = build_video_model(video_input_shape)
    text_inputs = tf.keras.Input(shape=text_feature_shape)

    combined_features = tf.keras.layers.concatenate([video_features, text_inputs])
    outputs = tf.keras.layers.Dense(3, activation='softmax')(combined_features)  # Assuming 3 classes for classification
    model = tf.keras.Model(inputs=[video_inputs, text_inputs], outputs=outputs)
    return model

# Example usage
video_input_shape = (N_FRAMES, HEIGHT, WIDTH, 3)
text_feature_shape = (MAX_TEXT_FEATURES,)

model = build_combined_model(video_input_shape, text_feature_shape)
model.summary()

# Learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10000,
    decay_rate=0.9
)

# Compile the model with the learning rate schedule
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# FrameGenerator class to preprocess video frames
class FrameGenerator:
    def __init__(self, paths, labels, features, n_frames, frame_size=(HEIGHT, WIDTH), training=False):
        self.paths = paths
        self.labels = labels
        self.features = features
        self.n_frames = n_frames
        self.frame_size = frame_size
        self.training = training

    def __call__(self):
        for video_path, label, feature in zip(self.paths, self.labels, self.features):
            cap = cv2.VideoCapture(video_path)
            frames = []
            while len(frames) < self.n_frames:
                ret, frame = cap.read()
                if not ret:
                    break
                frame = cv2.resize(frame, self.frame_size)  # Resize frames to smaller size
                frame = frame / 255.0  # Normalize
                frames.append(frame)
            cap.release()
            if len(frames) < self.n_frames:
                frames.extend([np.zeros(self.frame_size + (3,))] * (self.n_frames - len(frames)))  # Pad with zeros
            frames = np.array(frames)

            yield (frames, feature), label

output_signature = (
    (tf.TensorSpec(shape=(N_FRAMES, HEIGHT, WIDTH, 3), dtype=tf.float32), tf.TensorSpec(shape=(MAX_TEXT_FEATURES,), dtype=tf.float32)),
    tf.TensorSpec(shape=(), dtype=tf.int16)
)

def create_dataset(paths, labels, features, n_frames, batch_size, frame_size, training=False):
    dataset = tf.data.Dataset.from_generator(
        FrameGenerator(paths, labels, features, n_frames, frame_size, training=training),
        output_signature=output_signature
    )
    dataset = dataset.batch(batch_size)
    return dataset

# Assuming df is your DataFrame containing video paths, labels, and text features
# Ensure you have a DataFrame ready with these columns: 'Video Name', 'Label', 'Text_Features'
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_paths = train_df['Video Name'].tolist()
train_labels = train_df['Label'].tolist()
train_features = np.array(train_df['Text_Features'].tolist())

val_paths = test_df['Video Name'].tolist()
val_labels = test_df['Label'].tolist()
val_features = np.array(test_df['Text_Features'].tolist())

train_ds = create_dataset(train_paths, train_labels, train_features, N_FRAMES, BATCH_SIZE, frame_size=(HEIGHT, WIDTH), training=True)
val_ds = create_dataset(val_paths, val_labels, val_features, N_FRAMES, BATCH_SIZE, frame_size=(HEIGHT, WIDTH))

import gc
gc.collect()
if tf.config.list_physical_devices('GPU'):
    tf.keras.backend.clear_session()
    tf.compat.v1.reset_default_graph()

# Callbacks for early stopping and checkpointing
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True)
]

# Fit the model with callbacks
history = model.fit(train_ds, epochs=20, validation_data=val_ds, callbacks=callbacks)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 3, 112, 112, 3)]     0         []                            
                                                                                                  
 resize_video (ResizeVideo)  (None, 3, 112, 112, 3)       0         ['input_1[0][0]']             
                                                                                                  
 video_data_augmentation (V  (None, 3, 112, 112, 3)       0         ['resize_video[0][0]']        
 ideoDataAugmentation)                                                                            
                                                                                                  
 project (Project)           (None, 3, 112, 112, 32)      128       ['video_data_augmentation[

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


684/684 [==============================] - 623s 910ms/step - loss: 1.0350 - accuracy: 0.4518 - val_loss: 1.0846 - val_accuracy: 0.4561
Epoch 3/20
684/684 [==============================] - 644s 942ms/step - loss: 1.0062 - accuracy: 0.4751 - val_loss: 1.0365 - val_accuracy: 0.4561
Epoch 4/20
684/684 [==============================] - 624s 912ms/step - loss: 0.9899 - accuracy: 0.4971 - val_loss: 0.9903 - val_accuracy: 0.4561
Epoch 5/20
684/684 [==============================] - 648s 947ms/step - loss: 0.9779 - accuracy: 0.5307 - val_loss: 0.9954 - val_accuracy: 0.4561
Epoch 6/20
684/684 [==============================] - 640s 936ms/step - loss: 0.9694 - accuracy: 0.5482 - val_loss: 0.9655 - val_accuracy: 0.4561
Epoch 7/20
684/684 [==============================] - 651s 952ms/step - loss: 0.9576 - accuracy: 0.5804 - val_loss: 0.9572 - val_accuracy: 0.7778
Epoch 8/20
684/684 [==============================] - 642s 938ms/step - loss: 0.9495 - accuracy: 0.6067 - val_loss: 0.9515 - val_accura

In [21]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential
import einops
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Define dimensions
HEIGHT = 112
WIDTH = 112
N_FRAMES = 3
BATCH_SIZE = 2
MAX_TEXT_FEATURES = 3  # This should match the number of classes from the LLM

# Define a simple Conv3D block
def Conv3DBlock(filters, kernel_size):
    return Sequential([
        layers.Conv3D(filters, kernel_size, activation='relu', padding='same'),
        layers.MaxPooling3D(pool_size=(1, 2, 2))
    ])

# Video input model
def build_video_model(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = ResizeVideo(HEIGHT, WIDTH)(inputs)
    x = Conv3DBlock(32, (3, 3, 3))(x)
    x = Conv3DBlock(64, (3, 3, 3))(x)
    x = layers.GlobalAveragePooling3D()(x)
    return inputs, x

# Combined model
def build_combined_model(video_input_shape, text_feature_shape):
    video_inputs, video_features = build_video_model(video_input_shape)
    text_inputs = tf.keras.Input(shape=text_feature_shape)

    combined_features = tf.keras.layers.concatenate([video_features, text_inputs])
    outputs = tf.keras.layers.Dense(3, activation='softmax')(combined_features)  # Assuming 3 classes for classification
    model = tf.keras.Model(inputs=[video_inputs, text_inputs], outputs=outputs)
    return model

# Example usage
video_input_shape = (N_FRAMES, HEIGHT, WIDTH, 3)
text_feature_shape = (MAX_TEXT_FEATURES,)

model = build_combined_model(video_input_shape, text_feature_shape)
model.summary()

# Compile the model with a constant learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# FrameGenerator class to preprocess video frames
class FrameGenerator:
    def __init__(self, paths, labels, features, n_frames, frame_size=(HEIGHT, WIDTH), training=False):
        self.paths = paths
        self.labels = labels
        self.features = features
        self.n_frames = n_frames
        self.frame_size = frame_size
        self.training = training

    def __call__(self):
        for video_path, label, feature in zip(self.paths, self.labels, self.features):
            cap = cv2.VideoCapture(video_path)
            frames = []
            while len(frames) < self.n_frames:
                ret, frame = cap.read()
                if not ret:
                    break
                frame = cv2.resize(frame, self.frame_size)  # Resize frames to smaller size
                frame = frame / 255.0  # Normalize
                frames.append(frame)
            cap.release()
            if len(frames) < self.n_frames:
                frames.extend([np.zeros(self.frame_size + (3,))] * (self.n_frames - len(frames)))  # Pad with zeros
            frames = np.array(frames)

            yield (frames, feature), label

output_signature = (
    (tf.TensorSpec(shape=(N_FRAMES, HEIGHT, WIDTH, 3), dtype=tf.float32), tf.TensorSpec(shape=(MAX_TEXT_FEATURES,), dtype=tf.float32)),
    tf.TensorSpec(shape=(), dtype=tf.int16)
)

def create_dataset(paths, labels, features, n_frames, batch_size, frame_size, training=False):
    dataset = tf.data.Dataset.from_generator(
        FrameGenerator(paths, labels, features, n_frames, frame_size, training=training),
        output_signature=output_signature
    )
    dataset = dataset.batch(batch_size)
    return dataset

# Assuming df is your DataFrame containing video paths, labels, and text features
# Ensure you have a DataFrame ready with these columns: 'Video Name', 'Label', 'Text_Features'
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_paths = train_df['Video Name'].tolist()
train_labels = train_df['Label'].tolist()
train_features = np.array(train_df['Text_Features'].tolist())

val_paths = test_df['Video Name'].tolist()
val_labels = test_df['Label'].tolist()
val_features = np.array(test_df['Text_Features'].tolist())

train_ds = create_dataset(train_paths, train_labels, train_features, N_FRAMES, BATCH_SIZE, frame_size=(HEIGHT, WIDTH), training=True)
val_ds = create_dataset(val_paths, val_labels, val_features, N_FRAMES, BATCH_SIZE, frame_size=(HEIGHT, WIDTH))

# Fit the model
history = model.fit(train_ds, epochs=20, validation_data=val_ds)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 3, 112, 112, 3)]     0         []                            
                                                                                                  
 resize_video_2 (ResizeVide  (None, 3, 112, 112, 3)       0         ['input_5[0][0]']             
 o)                                                                                               
                                                                                                  
 sequential_16 (Sequential)  (None, 3, 56, 56, 32)        2624      ['resize_video_2[0][0]']      
                                                                                                  
 sequential_17 (Sequential)  (None, 3, 28, 28, 64)        55360     ['sequential_16[0][0]'] 

2024-08-03 22:25:33.950649: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 65028096 exceeds 10% of free system memory.
2024-08-03 22:25:33.950916: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 65028096 exceeds 10% of free system memory.


      1/Unknown - 1s 1s/step - loss: 1.1119 - accuracy: 0.0000e+00

2024-08-03 22:25:34.698226: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 65028096 exceeds 10% of free system memory.
2024-08-03 22:25:34.698457: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 65028096 exceeds 10% of free system memory.


      2/Unknown - 2s 730ms/step - loss: 1.1179 - accuracy: 0.0000e+00

2024-08-03 22:25:35.432408: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 65028096 exceeds 10% of free system memory.


342/342 [==============================] - 290s 847ms/step - loss: 1.0720 - accuracy: 0.4386 - val_loss: 1.0466 - val_accuracy: 0.4035
Epoch 2/20
342/342 [==============================] - 289s 847ms/step - loss: 1.0221 - accuracy: 0.5322 - val_loss: 1.0162 - val_accuracy: 0.4094
Epoch 3/20
342/342 [==============================] - 289s 844ms/step - loss: 0.9957 - accuracy: 0.5175 - val_loss: 1.0001 - val_accuracy: 0.4211
Epoch 4/20
342/342 [==============================] - 283s 825ms/step - loss: 0.9804 - accuracy: 0.5307 - val_loss: 0.9905 - val_accuracy: 0.4620
Epoch 5/20
342/342 [==============================] - 295s 864ms/step - loss: 0.9704 - accuracy: 0.5526 - val_loss: 0.9841 - val_accuracy: 0.4678
Epoch 6/20
342/342 [==============================] - 286s 837ms/step - loss: 0.9631 - accuracy: 0.5775 - val_loss: 0.9792 - val_accuracy: 0.5088
Epoch 7/20
342/342 [==============================] - 292s 854ms/step - loss: 0.9572 - accuracy: 0.6023 - val_loss: 0.9750 - val_accura

In [34]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential
import einops
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Define dimensions
HEIGHT = 112
WIDTH = 112
N_FRAMES = 3
BATCH_SIZE = 1
MAX_TEXT_FEATURES = 3  # This should match the number of classes from the LLM

# Define Conv2Plus1D layer
class Conv2Plus1D(layers.Layer):
    def __init__(self, filters, kernel_size, padding):
        super().__init__()
        self.seq = Sequential([
            layers.Conv3D(filters=filters, kernel_size=(1, kernel_size[1], kernel_size[2]), padding=padding),
            layers.Conv3D(filters=filters, kernel_size=(kernel_size[0], 1, 1), padding=padding)
        ])

    def call(self, x):
        return self.seq(x)

# Define Project layer
class Project(layers.Layer):
    def __init__(self, filters):
        super().__init__()
        self.conv = layers.Conv3D(filters, kernel_size=1)

    def call(self, x):
        return self.conv(x)

# Define ResidualMain layer
class ResidualMain(layers.Layer):
    def __init__(self, filters, kernel_size, dropout_rate=0.3, l2_strength=0.01):
        super().__init__()
        self.seq = Sequential([
            Conv2Plus1D(filters=filters, kernel_size=kernel_size, padding='same'),
            layers.LayerNormalization(),
            layers.ReLU(),
            layers.Dropout(dropout_rate),
            Conv2Plus1D(filters=filters, kernel_size=kernel_size, padding='same'),
            layers.LayerNormalization(),
            layers.Dropout(dropout_rate)
        ])

    def call(self, x):
        return self.seq(x)

# Add residual block
def add_residual_block(input, filters, kernel_size):
    out = ResidualMain(filters, kernel_size)(input)
    res = input
    if out.shape[-1] != input.shape[-1]:
        res = Project(out.shape[-1])(res)
    return layers.add([res, out])

# Resize video frames
class ResizeVideo(layers.Layer):
    def __init__(self, height, width):
        super().__init__()
        self.height = height
        self.width = width
        self.resizing_layer = layers.Resizing(self.height, self.width)

    def call(self, video):
        old_shape = einops.parse_shape(video, 'b t h w c')
        images = einops.rearrange(video, 'b t h w c -> (b t) h w c')
        images = self.resizing_layer(images)
        videos = einops.rearrange(images, '(b t) h w c -> b t h w c', t=old_shape['t'])
        return videos

# Data augmentation for video frames
class VideoDataAugmentation(layers.Layer):
    def __init__(self, frame_size):
        super().__init__()
        self.frame_size = frame_size
        self.augmentation_layer = Sequential([
            layers.RandomFlip("horizontal"),
            layers.RandomRotation(0.1),
            layers.RandomZoom(0.1),
            layers.RandomCrop(self.frame_size[0], self.frame_size[1])
        ])

    def call(self, video):
        old_shape = einops.parse_shape(video, 'b t h w c')
        images = einops.rearrange(video, 'b t h w c -> (b t) h w c')
        images = self.augmentation_layer(images)
        videos = einops.rearrange(images, '(b t) h w c -> b t h w c', t=old_shape['t'])
        return videos

# Video input model
def build_video_model(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = ResizeVideo(HEIGHT, WIDTH)(inputs)
    x = VideoDataAugmentation((HEIGHT, WIDTH))(x)  # Apply augmentation
    x = add_residual_block(x, filters=32, kernel_size=(3, 3, 3))
    x = layers.MaxPooling3D(pool_size=(1, 2, 2))(x)
    x = add_residual_block(x, filters=64, kernel_size=(3, 3, 3))
    x = layers.MaxPooling3D(pool_size=(1, 2, 2))(x)
    x = add_residual_block(x, filters=128, kernel_size=(3, 3, 3))  # New block
    x = layers.GlobalAveragePooling3D()(x)
    return inputs, x

# Combined model
def build_combined_model(video_input_shape, text_feature_shape):
    video_inputs, video_features = build_video_model(video_input_shape)
    text_inputs = tf.keras.Input(shape=text_feature_shape)

    combined_features = tf.keras.layers.concatenate([video_features, text_inputs])
    outputs = tf.keras.layers.Dense(3, activation='softmax')(combined_features)  # Assuming 3 classes for classification
    model = tf.keras.Model(inputs=[video_inputs, text_inputs], outputs=outputs)
    return model

# Example usage
video_input_shape = (N_FRAMES, HEIGHT, WIDTH, 3)
text_feature_shape = (MAX_TEXT_FEATURES,)

model = build_combined_model(video_input_shape, text_feature_shape)
model.summary()

# Learning rate scheduler
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10000,
    decay_rate=0.95
)

# Compile the model with the learning rate schedule
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# FrameGenerator class to preprocess video frames
class FrameGenerator:
    def __init__(self, paths, labels, features, n_frames, frame_size=(HEIGHT, WIDTH), training=False):
        self.paths = paths
        self.labels = labels
        self.features = features
        self.n_frames = n_frames
        self.frame_size = frame_size
        self.training = training

    def __call__(self):
        for video_path, label, feature in zip(self.paths, self.labels, self.features):
            cap = cv2.VideoCapture(video_path)
            frames = []
            while len(frames) < self.n_frames:
                ret, frame = cap.read()
                if not ret:
                    break
                frame = cv2.resize(frame, self.frame_size)  # Resize frames to smaller size
                frame = frame / 255.0  # Normalize
                frames.append(frame)
            cap.release()
            if len(frames) < self.n_frames:
                frames.extend([np.zeros(self.frame_size + (3,))] * (self.n_frames - len(frames)))  # Pad with zeros
            frames = np.array(frames)

            yield (frames, feature), label

output_signature = (
    (tf.TensorSpec(shape=(N_FRAMES, HEIGHT, WIDTH, 3), dtype=tf.float32), tf.TensorSpec(shape=(MAX_TEXT_FEATURES,), dtype=tf.float32)),
    tf.TensorSpec(shape=(), dtype=tf.int16)
)

def create_dataset(paths, labels, features, n_frames, batch_size, frame_size, training=False):
    dataset = tf.data.Dataset.from_generator(
        FrameGenerator(paths, labels, features, n_frames, frame_size, training=training),
        output_signature=output_signature
    )
    dataset = dataset.batch(batch_size)
    return dataset

# Assuming df is your DataFrame containing video paths, labels, and text features
# Ensure you have a DataFrame ready with these columns: 'Video Name', 'Label', 'Text_Features'
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_paths = train_df['Video Name'].tolist()
train_labels = train_df['Label'].tolist()
train_features = np.array(train_df['Text_Features'].tolist())

val_paths = test_df['Video Name'].tolist()
val_labels = test_df['Label'].tolist()
val_features = np.array(test_df['Text_Features'].tolist())

train_ds = create_dataset(train_paths, train_labels, train_features, N_FRAMES, BATCH_SIZE, frame_size=(HEIGHT, WIDTH), training=True)
val_ds = create_dataset(val_paths, val_labels, val_features, N_FRAMES, BATCH_SIZE, frame_size=(HEIGHT, WIDTH))

import gc
gc.collect()
if tf.config.list_physical_devices('GPU'):
    tf.keras.backend.clear_session()
    tf.compat.v1.reset_default_graph()

# Callbacks for early stopping and checkpointing
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True)
]

# Fit the model with callbacks
history = model.fit(train_ds, epochs=10, validation_data=val_ds, callbacks=callbacks)

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 3, 112, 112, 3)]     0         []                            
                                                                                                  
 resize_video_4 (ResizeVide  (None, 3, 112, 112, 3)       0         ['input_9[0][0]']             
 o)                                                                                               
                                                                                                  
 video_data_augmentation_2   (None, 3, 112, 112, 3)       0         ['resize_video_4[0][0]']      
 (VideoDataAugmentation)                                                                          
                                                                                            

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


684/684 [==============================] - 565s 826ms/step - loss: 1.0481 - accuracy: 0.4459 - val_loss: 1.0853 - val_accuracy: 0.4561
Epoch 3/10
684/684 [==============================] - 575s 840ms/step - loss: 1.0256 - accuracy: 0.4240 - val_loss: 1.0346 - val_accuracy: 0.4561
Epoch 4/10
684/684 [==============================] - 580s 848ms/step - loss: 1.0172 - accuracy: 0.4298 - val_loss: 1.0245 - val_accuracy: 0.4561
Epoch 5/10
684/684 [==============================] - 591s 864ms/step - loss: 1.0089 - accuracy: 0.4342 - val_loss: 1.0099 - val_accuracy: 0.4561
Epoch 6/10
684/684 [==============================] - 568s 830ms/step - loss: 1.0047 - accuracy: 0.4547 - val_loss: 1.0358 - val_accuracy: 0.4561
Epoch 7/10
684/684 [==============================] - 565s 826ms/step - loss: 1.0010 - accuracy: 0.4474 - val_loss: 1.0008 - val_accuracy: 0.4561
Epoch 8/10
684/684 [==============================] - 572s 836ms/step - loss: 0.9958 - accuracy: 0.4488 - val_loss: 0.9990 - val_accura

In [38]:
import tensorflow as tf
from tensorflow.keras import layers, Sequential
import einops
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Define dimensions
HEIGHT = 112
WIDTH = 112
N_FRAMES = 3
BATCH_SIZE = 4  # Increased batch size for better gradient estimation
MAX_TEXT_FEATURES = 3

# Define Conv2Plus1D layer with Batch Normalization
class Conv2Plus1D(layers.Layer):
    def __init__(self, filters, kernel_size, padding, dropout_rate=0.3):
        super().__init__()
        self.seq = Sequential([
            layers.Conv3D(filters=filters, kernel_size=(1, kernel_size[1], kernel_size[2]), padding=padding),
            layers.BatchNormalization(),  # Batch normalization for stabilization
            layers.ReLU(),
            layers.Conv3D(filters=filters, kernel_size=(kernel_size[0], 1, 1), padding=padding),
            layers.BatchNormalization(),
            layers.ReLU(),
            layers.Dropout(dropout_rate)  # Dropout for regularization
        ])

    def call(self, x):
        return self.seq(x)

# Define Project layer
class Project(layers.Layer):
    def __init__(self, filters):
        super().__init__()
        self.conv = layers.Conv3D(filters, kernel_size=1)

    def call(self, x):
        return self.conv(x)

# Define ResidualMain layer with L2 regularization
class ResidualMain(layers.Layer):
    def __init__(self, filters, kernel_size, dropout_rate=0.3, l2_strength=1e-4):
        super().__init__()
        self.seq = Sequential([
            Conv2Plus1D(filters=filters, kernel_size=kernel_size, padding='same', dropout_rate=dropout_rate),
            layers.Conv3D(filters=filters, kernel_size=(1, 1, 1), padding='same', kernel_regularizer=tf.keras.regularizers.l2(l2_strength))
        ])

    def call(self, x):
        return self.seq(x)

# Add residual block with projection for skip connection
def add_residual_block(input, filters, kernel_size):
    out = ResidualMain(filters, kernel_size)(input)
    res = input
    if out.shape[-1] != input.shape[-1]:
        res = Project(out.shape[-1])(res)
    return layers.add([res, out])

# Resize video frames
class ResizeVideo(layers.Layer):
    def __init__(self, height, width):
        super().__init__()
        self.height = height
        self.width = width
        self.resizing_layer = layers.Resizing(self.height, self.width)

    def call(self, video):
        old_shape = einops.parse_shape(video, 'b t h w c')
        images = einops.rearrange(video, 'b t h w c -> (b t) h w c')
        images = self.resizing_layer(images)
        videos = einops.rearrange(images, '(b t) h w c -> b t h w c', t=old_shape['t'])
        return videos

# Data augmentation for video frames
class VideoDataAugmentation(layers.Layer):
    def __init__(self, frame_size):
        super().__init__()
        self.frame_size = frame_size
        self.augmentation_layer = Sequential([
            layers.RandomFlip("horizontal"),
            layers.RandomRotation(0.1),
            layers.RandomZoom(0.1),
            layers.RandomCrop(self.frame_size[0], self.frame_size[1])
        ])

    def call(self, video):
        old_shape = einops.parse_shape(video, 'b t h w c')
        images = einops.rearrange(video, 'b t h w c -> (b t) h w c')
        images = self.augmentation_layer(images)
        videos = einops.rearrange(images, '(b t) h w c -> b t h w c', t=old_shape['t'])
        return videos

# Video input model
def build_video_model(input_shape):
    inputs = tf.keras.Input(shape=input_shape)
    x = ResizeVideo(HEIGHT, WIDTH)(inputs)
    x = VideoDataAugmentation((HEIGHT, WIDTH))(x)
    x = add_residual_block(x, filters=32, kernel_size=(3, 3, 3))
    x = layers.MaxPooling3D(pool_size=(1, 2, 2))(x)
    x = add_residual_block(x, filters=64, kernel_size=(3, 3, 3))
    x = layers.MaxPooling3D(pool_size=(1, 2, 2))(x)
    x = add_residual_block(x, filters=128, kernel_size=(3, 3, 3))
    x = layers.GlobalAveragePooling3D()(x)
    return inputs, x

# Combined model
def build_combined_model(video_input_shape, text_feature_shape):
    video_inputs, video_features = build_video_model(video_input_shape)
    text_inputs = tf.keras.Input(shape=text_feature_shape)

    combined_features = tf.keras.layers.concatenate([video_features, text_inputs])
    outputs = tf.keras.layers.Dense(3, activation='softmax')(combined_features)  # Assuming 3 classes for classification
    model = tf.keras.Model(inputs=[video_inputs, text_inputs], outputs=outputs)
    return model

# Example usage
video_input_shape = (N_FRAMES, HEIGHT, WIDTH, 3)
text_feature_shape = (MAX_TEXT_FEATURES,)

model = build_combined_model(video_input_shape, text_feature_shape)
model.summary()

# Learning rate scheduler with reduced decay
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10000,
    decay_rate=0.95
)

# Compile the model with the learning rate schedule
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# FrameGenerator class to preprocess video frames
class FrameGenerator:
    def __init__(self, paths, labels, features, n_frames, frame_size=(HEIGHT, WIDTH), training=False):
        self.paths = paths
        self.labels = labels
        self.features = features
        self.n_frames = n_frames
        self.frame_size = frame_size
        self.training = training

    def __call__(self):
        for video_path, label, feature in zip(self.paths, self.labels, self.features):
            cap = cv2.VideoCapture(video_path)
            frames = []
            while len(frames) < self.n_frames:
                ret, frame = cap.read()
                if not ret:
                    break
                frame = cv2.resize(frame, self.frame_size)
                frame = frame / 255.0
                frames.append(frame)
            cap.release()
            if len(frames) < self.n_frames:
                frames.extend([np.zeros(self.frame_size + (3,))] * (self.n_frames - len(frames)))
            frames = np.array(frames)

            yield (frames, feature), label

output_signature = (
    (tf.TensorSpec(shape=(N_FRAMES, HEIGHT, WIDTH, 3), dtype=tf.float32), tf.TensorSpec(shape=(MAX_TEXT_FEATURES,), dtype=tf.float32)),
    tf.TensorSpec(shape=(), dtype=tf.int16)
)

def create_dataset(paths, labels, features, n_frames, batch_size, frame_size, training=False):
    dataset = tf.data.Dataset.from_generator(
        FrameGenerator(paths, labels, features, n_frames, frame_size, training=training),
        output_signature=output_signature
    )
    dataset = dataset.batch(batch_size)
    return dataset

# Assuming df is your DataFrame containing video paths, labels, and text features
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_paths = train_df['Video Name'].tolist()
train_labels = train_df['Label'].tolist()
train_features = np.array(train_df['Text_Features'].tolist())

val_paths = test_df['Video Name'].tolist()
val_labels = test_df['Label'].tolist()
val_features = np.array(test_df['Text_Features'].tolist())

train_ds = create_dataset(train_paths, train_labels, train_features, N_FRAMES, BATCH_SIZE, frame_size=(HEIGHT, WIDTH), training=True)
val_ds = create_dataset(val_paths, val_labels, val_features, N_FRAMES, BATCH_SIZE, frame_size=(HEIGHT, WIDTH))

# Callbacks for early stopping and checkpointing
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=7, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True)
]

# Fit the model with callbacks
history = model.fit(train_ds, epochs=30, validation_data=val_ds, callbacks=callbacks)

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_13 (InputLayer)       [(None, 3, 112, 112, 3)]     0         []                            
                                                                                                  
 resize_video_6 (ResizeVide  (None, 3, 112, 112, 3)       0         ['input_13[0][0]']            
 o)                                                                                               
                                                                                                  
 video_data_augmentation_4   (None, 3, 112, 112, 3)       0         ['resize_video_6[0][0]']      
 (VideoDataAugmentation)                                                                          
                                                                                            

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


171/171 [==============================] - 410s 2s/step - loss: 1.0304 - accuracy: 0.4371 - val_loss: 1.0432 - val_accuracy: 0.4561
Epoch 3/30
171/171 [==============================] - 406s 2s/step - loss: 1.0211 - accuracy: 0.4503 - val_loss: 1.0178 - val_accuracy: 0.4561
Epoch 4/30
171/171 [==============================] - 410s 2s/step - loss: 1.0157 - accuracy: 0.4357 - val_loss: 1.0181 - val_accuracy: 0.6199
Epoch 5/30
171/171 [==============================] - 402s 2s/step - loss: 1.0115 - accuracy: 0.4342 - val_loss: 2.5929 - val_accuracy: 0.4035
Epoch 6/30
171/171 [==============================] - 406s 2s/step - loss: 1.0189 - accuracy: 0.4371 - val_loss: 1.1146 - val_accuracy: 0.4561
Epoch 7/30
171/171 [==============================] - 404s 2s/step - loss: 1.0074 - accuracy: 0.4357 - val_loss: 1.0239 - val_accuracy: 0.6082
Epoch 8/30
171/171 [==============================] - 418s 2s/step - loss: 1.0048 - accuracy: 0.4576 - val_loss: 1.0321 - val_accuracy: 0.5789
Epoch 9/30

# Saving the Model

In [16]:
import torch

In [25]:
model_path = 'hybrid_model.pth'
torch.save(model.state_dict(), model_path)
# Move the model to the appropriate device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loaded_model.to(device)
print(f"Model loaded from {model_path}")

AttributeError: 'Functional' object has no attribute 'state_dict'

In [26]:
import torch
from transformers import AutoModelForSequenceClassification

# Load and initialize the model
model_name = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Save the model's state dictionary
model_path = 'hybrid_model.pth'
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

# Load the model for inference or further training
loaded_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Make sure the model path exists before loading
try:
    loaded_model.load_state_dict(torch.load(model_path))
    print(f"Model loaded from {model_path}")
except FileNotFoundError as e:
    print(f"Error loading model: {e}")

# Move the model to the appropriate device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loaded_model.to(device)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model saved to hybrid_model.pth


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded from hybrid_model.pth


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [27]:
import os

save_directory = './models/'
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

model_path = os.path.join(save_directory, 'hybrid_model.pth')
torch.save(model.state_dict(), model_path)